In [2]:
from collections import defaultdict

def generate_parse_table(grammar, first, follow):
    table = defaultdict(dict)

    # Ensure all terminal symbols have their own first sets
    terminals = set()
    for rules in grammar.values():
        for prod in rules:
            for symbol in prod:
                if symbol not in grammar and symbol != '@':
                    terminals.add(symbol)

    for t in terminals:
        if t not in first:
            first[t] = {t}

    first['@'] = {'@'}  # Add epsilon explicitly

    for head in grammar:
        for production in grammar[head]:
            symbols = list(production)
            first_set = set()

            for symbol in symbols:
                symbol_first = first.get(symbol, {symbol})
                first_set |= symbol_first - {'@'}
                if '@' not in symbol_first:
                    break
            else:
                first_set.add('@')

            for terminal in first_set - {'@'}:
                table[head][terminal] = production

            if '@' in first_set:
                for terminal in follow[head]:
                    table[head][terminal] = production

    return table


# Sample grammar, first and follow sets
grammar = {
    'S': ['(L)', 'a'],
    'L': ['SQ'],
    'Q': [',SQ', '@']
}

first = {
    'S': {'(', 'a'},
    'L': {'(', 'a'},
    'Q': {',', '@'}
}

follow = {
    'S': {'$', ',', ')'},
    'L': {')'},
    'Q': {')'}
}

# Generate and print the parse table
parse_table = generate_parse_table(grammar, first, follow)

for non_terminal in sorted(parse_table):
    for terminal in sorted(parse_table[non_terminal]):
        print(f"[{non_terminal}][{terminal}] = {parse_table[non_terminal][terminal]}")


[L][(] = SQ
[L][a] = SQ
[Q][)] = @
[Q][,] = ,SQ
[S][(] = (L)
[S][a] = a
